# Movie-Studio-EDA-Project

In [3]:
import sqlite3
import pandas as pd

# Connect to IMDb database
imdb_conn = sqlite3.connect("./zippedData/im.db")

# List tables in IMDb database
imdb_tables= pd.read_sql("SELECT * FROM sqlite_master;", imdb_conn)
imdb_tables


,type,name,tbl_name,rootpage,sql
